## Problema

In [2]:
import Data.List (find)

type Nombre = String
type NumeroCuenta = Integer

data Estudiante = Estudiante { nombre :: Nombre
                             , cuenta :: NumeroCuenta
                             } deriving Show

data Calificacion = Calificacion { estudiante :: NumeroCuenta
                                 , promedio :: Double
                                 } deriving Show

type TablaEstudiantes = [Estudiante]
type TablaCalificaciones = [Calificacion]

estudiantes :: TablaEstudiantes
estudiantes = [ Estudiante "Canek"   1234
              , Estudiante "Galaviz" 2341
              , Estudiante "Lourdes" 3412
              , Estudiante "Urrutia" 4123
              ]

calificaciones :: TablaCalificaciones
calificaciones = [ Calificacion 1234 5.0
                 , Calificacion 2341 8.5
                 , Calificacion 3412 10.0
                 , Calificacion 4123 7.4]

buscaEstudiante :: Nombre -> Maybe Estudiante
buscaEstudiante nombreE = find ((== nombreE) . nombre) estudiantes

buscaCalificacion :: NumeroCuenta -> Maybe Calificacion
buscaCalificacion numCuenta = find ((== numCuenta) . estudiante) calificaciones

In [8]:
import Data.Maybe (fromJust)

-- fromJust :: Maybe a -> a
-- fromJust (Just x) = x
-- fromJust Nothing  = error "Oh no!"

-- Crea una función que recibe el nombre de un estudiante
-- y regresa si aprobó.
aprobado :: Nombre -> Maybe Bool
aprobado nombreE = Just
                    $ (>= 6)
                    $ promedio
                    $ fromJust
                    $ buscaCalificacion
                    $ cuenta
                    $ fromJust
                    $ buscaEstudiante nombreE
                    
aprobado' nombreE = fmap ((>=6) . promedio)
                    $ buscaCalificacion
                    $ cuenta
                    $ fromJust
                    $ buscaEstudiante nombreE

In [10]:
--aprobado' "Canek"
--aprobado' "Urrutia"
--aprobado' "Lourdes"
--aprobado' "Galaviz"
--aprobado' "JP"

aprobado "Canek"
aprobado "Urrutia"
aprobado "Lourdes"
aprobado "Galaviz"
aprobado "JP"

Just False

Just True

Just True

Just True

: 

# Mónadas

Tenemos:
- Un elemento en un contexto: `Just 5`, `[1,2,3]`, ...
- Una función que toma un elemento y te devuelve un elemento dentro de un contexto.

Deseamos lograr lo siguiente:

1. Sacar el elemento de su contexto.
2. Aplicarle la función que nos da al elemento en otro contexto.

## Detalles de implementación.

- Una mónada siempre es un Aplicativo.
- Queremos una función `return` que "eleve" un elemento. Es decir, dado un elemento, lo debe envolver en una mónada.
- Queremos una función `>>=` (llamada "bind") que implemente el comportamiento de las mónadas.
- La función _bind_ será infija, con asociatividad izquierda y jerarquía 1.
- Queremos una función `>>` (llamada "then" o "sequence") que dadas 2 mónadas, regrese la segunda con un comportamiento consistente con el de `>>=`.
- `>>` asocia a la izquierda y tiene jerarquía 1.

In [12]:
class Applicative m => Monad m where
    return :: a -> m a
    return = pure
    
    infixl 1 >>=
    (>>=) :: m a -> (a -> m b) -> m b
    
    infixl 1 >>
    (>>) :: m a -> m b -> m b
    ma >> mb = ma >>= (\_ -> mb)

Line 10: Use >>
Found:
ma >>= (\ _ -> mb)
Why not:
ma >> mbLine 10: Use const
Found:
\ _ -> mb
Why not:
const mb

In [15]:
:i Monad

type Monad :: (* -> *) -> Constraint
class Applicative m => Monad m where
  return :: a -> m a
  (>>=) :: m a -> (a -> m b) -> m b
  (>>) :: m a -> m b -> m b
  {-# MINIMAL (>>=) #-}
  	-- Defined at <interactive>:1:1

Define al tipo `Maybe` como instancia de mónadas.

In [17]:
-- data Maybe a = Nothing | Just a

instance Monad Maybe where
    Nothing  >>= _ = Nothing
    (Just x) >>= f = f x

In [18]:
Just 5 >>= (\x -> Just (x + 1))
Nothing >>= (\x -> Just (x + 1))

Line 2: Evaluate
Found:
Nothing >>= (\ x -> Just (x + 1))
Why not:
Nothing

Just 6

Nothing

Define a las listas como instancias de mónadas.

In [25]:
instance Monad [] where
    xs >>= f = [y | x <- xs, y <- f x]

In [26]:
[1..5] >>= (\x -> replicate x x)
--fmap (\x -> replicate x x) [1..5]

[1,2,2,3,3,3,4,4,4,4,5,5,5,5,5]

### Ejercicio

Modifica el ejercicio original, ahora utilizando mónadas.

In [29]:
--(>>=) :: m a -> (a -> m b) -> m b
--(>>=) :: Maybe Integer -> (Integer -> Maybe Calificacion) -> Maybe Calificacion

-- cuenta :: Estudiante -> NumeroCuenta
-- buscaCalificacion :: NumeroCuenta -> Maybe Calificacion
-- buscaCalificacion . cuenta :: Estudiante -> Maybe Calificacion

aprobado' :: Nombre -> Maybe Bool
aprobado' nombreE = buscaEstudiante nombreE
                    >>= (buscaCalificacion . cuenta)
                    >>= (\c -> return (promedio c >= 6))

In [30]:
aprobado' "Canek"
aprobado' "Urrutia"
aprobado' "Lourdes"
aprobado' "Galaviz"
aprobado' "JP"

Just False

Just True

Just True

Just True

Nothing

## Notación `do`

Cuando trabajamos con mónadas, es muy común encontrar situaciones como la siguiente:

```haskell
monada1 >>= (
    \x1 -> f x1 >>= (
        \x2 -> g x2 >>= ...))
```

Haskell nos da una alternativa:

```haskell
do
    x1 <- monada1
    x2 <- f x1
    ...
```

Cuando utilizamos el operador `>>` es equivalente a no asignar nada:

```haskell
monada1 >> monada2

do
    monada1
    monada2
```

Igual podemos utilizar `return`:

```haskell
do
    ...
    return resultado
```

### Ejercicio

Modifica el ejercicio anterior para utilizar la notación `do`.

In [32]:
aprobado' :: Nombre -> Maybe Bool
aprobado' nombreE = buscaEstudiante nombreE
                    >>= (\e -> buscaCalificacion $ cuenta e)
                    >>= (\c -> return (promedio c >= 6))

aprobado'' :: Nombre -> Maybe Bool
aprobado'' nombreE = do
                        e <- buscaEstudiante nombreE
                        c <- buscaCalificacion (cuenta e)
                        return (promedio c >= 6)

Line 3: Avoid lambda
Found:
\ e -> buscaCalificacion $ cuenta e
Why not:
buscaCalificacion . cuenta

In [33]:
aprobado'' "Canek"
aprobado'' "Urrutia"
aprobado'' "Lourdes"
aprobado'' "Galaviz"
aprobado'' "JP"

Just False

Just True

Just True

Just True

Nothing

## Leyes

- Identidad izquierda: `return a >>= f ≡ f a`.
- Identidad derecha: `m >>= return ≡ m`.
- Asociatividad: `(m >>= f) >>= g ≡ m >>= (\x -> f x >>= g)`